## Optimizing Model Parameters

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to available device
model.to(device)

Using device: mps


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# Initialize loss function
loss_fn = nn.CrossEntropyLoss()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Move data to device
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d} | {size:>5d}]")

def test_loop(dataloader, model, loss_fn, device):
    # Set model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating model with torch.no_grad() ensures that no gradients are computed during test mode
    # Also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            # Move data to device
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
for t in range(epochs):
    print(f"Epoch {t+1} \n--------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)
print("Done!")

Epoch 1 
--------------------
loss: 2.326514 [   64 | 60000]
loss: 2.302029 [ 6464 | 60000]
loss: 2.281895 [12864 | 60000]
loss: 2.264354 [19264 | 60000]
loss: 2.251766 [25664 | 60000]
loss: 2.231588 [32064 | 60000]
loss: 2.238654 [38464 | 60000]
loss: 2.207314 [44864 | 60000]
loss: 2.198253 [51264 | 60000]
loss: 2.163849 [57664 | 60000]
Test Error: 
 Accuracy: 42.6%, Avg loss: 2.161832 

Epoch 2 
--------------------
loss: 2.183395 [   64 | 60000]
loss: 2.158848 [ 6464 | 60000]
loss: 2.105554 [12864 | 60000]
loss: 2.115267 [19264 | 60000]
loss: 2.067223 [25664 | 60000]
loss: 2.019543 [32064 | 60000]
loss: 2.046306 [38464 | 60000]
loss: 1.966715 [44864 | 60000]
loss: 1.964195 [51264 | 60000]
loss: 1.902733 [57664 | 60000]
Test Error: 
 Accuracy: 53.8%, Avg loss: 1.897734 

Epoch 3 
--------------------
loss: 1.937932 [   64 | 60000]
loss: 1.893978 [ 6464 | 60000]
loss: 1.783830 [12864 | 60000]
loss: 1.821845 [19264 | 60000]
loss: 1.717768 [25664 | 60000]
loss: 1.674977 [32064 | 60000]
